In [ ]:
# chargement des données de playlist
import numpy as np
data = np.load("../resources/music_2.npy")
len(data)

100000

In [44]:
# récupération uniquement des identifiant de morceaux // suppression des identifiant d'artiste
playlist_track = [list(filter(lambda w: w.split("_")[0]==u"track",playlist)) for playlist in data]

In [45]:
# nombre de morceaux != ?
tracks = np.unique(np.concatenate(playlist_track))
Vt = len(tracks)
Vt

338509

In [46]:
# nombre d'occurence de chaque morceaux ?
track_counts = dict((tracks[i],0) for i in range(0, Vt))
for p in playlist_track:
    for a in p:
        track_counts[a]=track_counts[a]+1;

In [47]:
# filtrage des morceaux très peu fréquents
playlist_track_filter = [list(filter(lambda a : track_counts[a]> 7, playlist)) for playlist in playlist_track]
track_f = np.unique(np.concatenate(playlist_track_filter))
Vt = len(track_f)

In [48]:
# construction d'un dict id_morceaux id [0,Vt]
track_dict = dict((track_f[i],i) for i in range(0, Vt))
# conversion des playlisat en liste d'entier
corpus_num_track = [[track_dict[track] for track in play ] for play in playlist_track_filter]

In [49]:
# import de Keras
from keras.models import Sequential, Model
from keras.layers import Embedding, Reshape, Activation, Input, Dense
from keras.layers.merge import Dot
from keras.utils import np_utils
from keras.preprocessing.sequence import skipgrams

In [50]:
# hyper-paramètres de word2vec :
# dimension de l'espace latent
vector_dim = 50
# taille de la fenêtre de voisinage
window_width = 5
# sur-échantillonage des exemples négatifs
neg_sample = 4.
# taille des mini-batch
min_batch_size = 50
# coeff pour la loi de tirage des exemple negatif
samp_coef = -0.1

In [51]:
# comptage du nombre d'occurences des morceaux dans les playlist filtrées
tracks_counts_f = dict((track_f[i],0) for i in range(0, Vt))
for p in playlist_track_filter:
    for t in p:
        tracks_counts_f[t]=tracks_counts_f[t]+1;
# construction de la table de tirage des morceaux pour les exmeple negatif en utilisant ces fréquences
spt_tracks=np.array(list(map(lambda a:tracks_counts_f[a],track_f)),np.float)
sptn_tracks=np.power(spt_tracks,samp_coef)
sptn_tracks=sptn_tracks/np.sum(sptn_tracks)
sptn_tracks=np.cumsum(np.sort(sptn_tracks)[::-1])

In [52]:
# construction du réseau word2vec

# entrée deux entier (couple de morceaux)
input_target_t = Input((1,), dtype='int32')
input_context_t = Input((1,), dtype='int32')

# définition de l'embeding
embedding_t_t = Embedding(Vt, vector_dim, input_length=1, name='embedding_t')
# projection du premier morceau
target_t = embedding_t_t(input_target_t)
target_t = Reshape((vector_dim, 1))(target_t)

# projection du second morceaux
context_t = embedding_t_t(input_context_t)
context_t = Reshape((vector_dim, 1))(context_t)

# calcul de la sortie
dot_product_t = Dot(axes=0)([target_t, context_t])
dot_product_t = Reshape((1,))(dot_product_t)
output_t = Dense(1, activation='sigmoid',name="classif")(dot_product_t)

# definition du modèle
SkipGram_t = Model(inputs=[input_target_t, input_context_t], outputs=output_t)
SkipGram_t.compile(loss='binary_crossentropy', optimizer='adam')

In [53]:
# définition du générateur de couple de morceaux (y=0 <-> aléatoire, y=1 <-> proche dans une playlist)
import random
def track_ns_generator(corpus_num,nbm):
    while 1:
        Data=[]
        Labels=[]
        for i, doc in enumerate(random.sample(corpus_num,nbm)):
            data, labels = skipgrams(sequence=doc, vocabulary_size=Vt, window_size=window_width, negative_samples=neg_sample,sampling_table=sptn_tracks)
            if (len(data)>0):
                Data.append(np.array(data, dtype=np.int32))
                Labels.append(np.array(labels, dtype=np.int32))
        Data=np.concatenate(Data)
        Labels=np.concatenate(Labels)
        x=[Data[:,0],Data[:,1]]
        y=Labels
        yield (x,y)

In [54]:
# ensemble de test et d'apprentissage
index_tst = np.random.choice(100000,10000)
index_app  = np.setdiff1d(range(100000),index_tst)

play_app   = [corpus_num_track[i] for i in index_app]
play_tst  = [corpus_num_track[i] for i in index_tst]


In [55]:
# apprentissage
hist=SkipGram_t.fit_generator(track_ns_generator(play_app,min_batch_size),200,50)

Epoch 1/50
200/200 [==============================] - 9s 46ms/step - loss: 0.6657
Epoch 2/50
200/200 [==============================] - 8s 41ms/step - loss: 0.6177
Epoch 3/50
200/200 [==============================] - 9s 44ms/step - loss: 0.5563
Epoch 4/50
200/200 [==============================] - 9s 43ms/step - loss: 0.4887
Epoch 5/50
200/200 [==============================] - 9s 45ms/step - loss: 0.4827
Epoch 6/50
200/200 [==============================] - 9s 43ms/step - loss: 0.4837
Epoch 7/50
200/200 [==============================] - 8s 42ms/step - loss: 0.4819
Epoch 8/50
200/200 [==============================] - 9s 43ms/step - loss: 0.4862
Epoch 9/50
200/200 [==============================] - 8s 41ms/step - loss: 0.4820
Epoch 10/50
200/200 [==============================] - 9s 44ms/step - loss: 0.4833
Epoch 11/50
200/200 [==============================] - 9s 43ms/step - loss: 0.4852
Epoch 12/50
200/200 [==============================] - 9s 45ms/step - loss: 0.4853
Epoch 13/50
2

In [56]:
# récupérations des positions des morceaux dans l'espace de projection
vectors_tracks = SkipGram_t.get_weights()[0]

In [57]:
from sklearn.metrics import pairwise_distances

In [58]:
# fonction retpournant les indices des s plus proches voisins des seeds dans la matrice X
# a vous de jouer avec pairwise_distances, np.min, np.argsort,...
def predict(seeds,s,X, metric='euclidean'):
    V = X.shape[0]
    others = np.setdiff1d(range(V),seeds)
    D = pairwise_distances(X[seeds,:],X[others,:],metric)
    return others[np.argsort(np.min(D,0))[:s]]

In [59]:
# exemple pour 4 suggestions avec les 2 premiers morceaux en seeds 
pr=predict([0,1],4,vectors_tracks)
pr

array([893,  66, 504, 787])

In [19]:
# estimation des performances
# hit@10
# nombre de bonne prediction
goodpred = []
# nombre de predictions faites
nbpred   = []
# pour chaque playlist
for p in play_tst:
    # si au moins deux chansons
    if (len(p)>1):
        # recuperations des seeds 5 premiers morceaux ou moins si la playlist contient moins de 5 morceaux
        seeds  = p[:np.min([5,len(p)-1])]
        # recuperations de la suite de la playlist que nous allons comparer à nos suggestions
        topred = p[np.min([5,len(p)-1]):]
        # construction des suggestions 10 suggestions par morceaux a predire
        prediction = predict(seeds,10*len(topred),vectors_tracks)
        # comptage du nombre de morceaux présent dans nos suggestions
        goodpred.append(len(np.intersect1d(prediction,topred)))
        # stockage du nombre de predictions
        nbpred.append(len(topred))
# proportions de morceux présents dans nos suggestions
hitat10 = np.sum(goodpred)/np.sum(nbpred)

KeyboardInterrupt: 

In [ ]:
hitat10

# Test of the metrics for pairwise_distances

In [60]:
def test_pairwise(metrics='euclidean'):
    # nombre de bonne prediction
    goodpred = []
    # nombre de predictions faites
    nbpred   = []
    # pour chaque playlist
    for p in play_tst:
        # si au moins deux chansons
        if (len(p)>1):
            # recuperations des seeds 5 premiers morceaux ou moins si la playlist contient moins de 5 morceaux
            seeds  = p[:np.min([5,len(p)-1])]
            # recuperations de la suite de la playlist que nous allons comparer à nos suggestions
            topred = p[np.min([5,len(p)-1]):]
            # construction des suggestions 10 suggestions par morceaux a predire
            prediction = predict(seeds,10*len(topred),vectors_tracks, metrics)
            # comptage du nombre de morceaux présent dans nos suggestions
            goodpred.append(len(np.intersect1d(prediction,topred)))
            # stockage du nombre de predictions
            nbpred.append(len(topred))
    # proportions de morceux présents dans nos suggestions
    return np.sum(goodpred)/np.sum(nbpred)

In [ ]:
_VALID_METRICS = ['euclidean', 'cosine', 'manhattan']
hitat10_results = dict((metric, test_pairwise(metric)) for metric in _VALID_METRICS)
hitat10_results

## Bar chart for the results of hitat10

In [22]:
import operator
import matplotlib.pyplot as plt
sorted_hitat10 = sorted(hitat10_results.items(), key=operator.itemgetter(1))
sorted_hitat10

NameError: name 'hitat10_results' is not defined

In [ ]:
fig, ax = plt.subplots()

# Dataset
values = []
labels = []
for item in sorted_hitat10:
    values.append(item[1])
    labels.append(item[0])
    
# Create bars and choose color
y_pos = np.arange(len(labels))
plt.bar(y_pos, values, color = (0.5,0.1,0.5,0.6))

# Add title and axis names
plt.title('Hitat10 results for different metrics')
plt.xlabel('Metrics')
plt.ylabel('score')
 
# Limits for the Y axis
plt.ylim(0, np.max(values) * 1.5)
 
# Create names
plt.xticks(y_pos, labels)
 
# Show graphic
plt.show()

# A faire

- faire quelque graphique sen particulier pour discuter le choix de n'avoir garder que les morceaux présents dans plus de sept playlists
- coder la fonction predict permetant de trouver les morceaux les plus proches des seeds (dans un premier temps de manière simple en utilisant la fonction pairwise_dist de sklearn)
- tester l'influence des différents type de distance sur les les performances en terme de hit@10
- vous pourrez ensuite essayé d'optimiser cette recherche https://scikit-learn.org/stable/modules/neighbors.html
- faites varier un hyper-paramètres sur une plage raisonable 
- faire un graphique de hit@10 en fonction des valeurs de l'hyperparamètre
- faire de même avec un second
- proposer des valeures finales pour les hyper-paramètres
- joindre avec les meta-données, url des morceaux ....
- exporter // faire le liens avec le front-end

## Aide

In [ ]:
# aide jointure
import pandas as pd
tr_meta=pd.read_csv("resources/./Tracks_V2.csv")
jdf = pd.DataFrame({"id":track_f,"index":range(Vt)})
jdf["deezer_id"]=jdf["id"].apply(lambda x: float(x.split("_")[1]))

In [ ]:
trj_meta = tr_meta.merge(jdf, left_on="id",right_on="deezer_id")
trj_meta.set_index("index",inplace=True)

In [ ]:
trj_meta

# Implementation of predict_opt

In [140]:
#aide predict_opt
#créer un kd-tree
from sklearn.neighbors import KDTree
from operator import itemgetter

kdt = KDTree(vectors_tracks, leaf_size=30, metric='euclidean')

#fonction de recherche a compléter
#vérifer la cohérences des résulats avec predict
# TODO : enlever les premiers voisins (ils sont à une distance 0 car il s'agit des points eux-memes) 
def predict_opt(seeds,s,X,kdt):
    V = X.shape[0]
    seeds_size = len(seeds)
    others = np.setdiff1d(range(V),seeds)
    
    dist, ind = kdt.query(X[seeds , :], k = s + seeds_size)
    dist_ind_list = np.concatenate(np.dstack((dist, ind)))
    dist_ind_list2 = list(map(lambda x : tuple(x), dist_ind_list))
    sorted_dist_ind_list = sorted(dist_ind_list2, key=itemgetter(0))[seeds_size : (s + seeds_size)]

    neigh = list(map(lambda x : x[1], sorted_dist_ind_list))
    return neigh

In [145]:
# vérification de la cohérences des résultats des suggestion
pr=predict([0,1],25, vectors_tracks, 'euclidean')
pr

array([ 893,   66,  504,  787,  483,  789, 1452, 1828, 1383, 1546, 1436,
       3311, 1709, 2268, 1547, 2385, 2956, 1545, 1784, 1384, 1840, 2847,
        446, 3043, 1976])

In [146]:
pr2=predict_opt([0,1],25,vectors_tracks, kdt)
pr2

[893.0,
 66.0,
 504.0,
 787.0,
 483.0,
 789.0,
 1452.0,
 1828.0,
 1383.0,
 1546.0,
 1436.0,
 3311.0,
 1709.0,
 2268.0,
 1547.0,
 2385.0,
 2956.0,
 1545.0,
 1784.0,
 1384.0,
 1840.0,
 2847.0,
 446.0,
 3043.0,
 1976.0]

In [ ]:
# estimation des performances
# hit@10
# nombre de bonne prediction
goodpred = []
# nombre de predictions faites
nbpred   = []
# pour chaque playlist
for p in play_tst:
    # si au moins deux chansons
    if (len(p)>1):
        # recuperations des seeds 5 premiers morceaux ou moins si la playlist contient moins de 5 morceaux
        seeds  = p[:np.min([5,len(p)-1])]
        # recuperations de la suite de la playlist que nous allons comparer à nos suggestions
        topred = p[np.min([5,len(p)-1]):]
        # construction des suggestions 10 suggestions par morceaux a predire
        prediction = predict_opt(seeds,10*len(topred),vectors_tracks,kdt)
        # comptage du nombre de morceaux présent dans nos suggestions
        goodpred.append(len(np.intersect1d(prediction,topred)))
        # stockage du nombre de predictions
        nbpred.append(len(topred))
# proportions de morceux présents dans nos suggestions
hitat10 = np.sum(goodpred)/np.sum(nbpred)

# Testing hyper-parameters for KDTree

## Testing hyper-parameters 'leaf_size' for KDTree

In [64]:
def test_predict_opt(leaf_size = 10, metric = 'euclidean'):
    
    kdt = KDTree(vectors_tracks, leaf_size, metric)
    # nombre de bonne prediction
    goodpred = []
    # nombre de predictions faites
    nbpred   = []
    # pour chaque playlist
    for p in play_tst:
        # si au moins deux chansons
        if (len(p)>1):
            seeds  = p[:np.min([5,len(p)-1])]
            topred = p[np.min([5,len(p)-1]):]
            
            prediction = predict_opt(seeds,10*len(topred),vectors_tracks, kdt)
            goodpred.append(len(np.intersect1d(prediction,topred)))
            nbpred.append(len(topred))

    return np.sum(goodpred)/np.sum(nbpred)

## Testing hyper-parameter 'metric' for KDTree

In [ ]:
trj_meta.sort_values(by=['rank'],ascending=False)["title"].head(20)

In [ ]:
# les voisins de i will survive ?
b=predict([9152],4,vectors_tracks)
trj_meta.loc[b,"title"]

In [ ]:
# un peu de musique
import IPython
IPython.display.Audio(trj_meta.loc[19030,"preview"])